# Redes Neuronales

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
import plotly.graph_objects as go
import os
import pickle

2025-06-26 19:37:10.325032: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-26 19:37:10.529075: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-26 19:37:10.703365: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750959430.923213   13921 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750959430.981162   13921 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750959431.359585   13921 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

* Cargamos .csv y separamos "X" e "y" . Recordemos "X" son todas las columnas excepto la columna target en cambio "y" es la columna target.

In [2]:
df = pd.read_csv('datos_limpios_modelo_10cols.csv')


X = df.drop(columns=['precio_contado'])
y = df['precio_contado']


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


* Creamos un modelo de red neuronal con varias capas:
  - La primera capa recibe los datos de entrada (el numero de caracteristicas que tiene cada ejemplo).
  - Luego hay dos capas intermedias (ocultas) con varias neuronas que ayudan a que el modelo aprenda patrones complejos.
  - La ultima capa tiene una neurona que nos da la prediccion del precio.

* Preparamos el modelo para entrenarlo:
  - Usamos **Adam** para ajustar el modelo.
  - Le decimos que mida que tan bien esta haciendo la prediccion con  **MSE** (error cuadratico medio).
  - Tambien le pedimos que nos muestre **MAE** (error absoluto medio) para entender mejor su rendimiento.


In [3]:
model_mse = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)
])

model_mse.compile(optimizer=Adam(), loss='mse', metrics=['mae'])



2025-06-26 19:37:17.067564: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


* Entrenamos el modelo usando los datos de entrenamiento ("X_train" y "y_train").
* Tambien usamos los datos de prueba ("X_test" y "y_test") para validar como esta funcionando el modelo mientras aprende.
* Definimos que el entrenamiento dure 100 epocas (pasadas completas sobre todos los datos).
* En cada epoca, el modelo procesa los datos en grupos de 32 ejemplos (batch size).
* "verbose=1" hace que se muestre una barra de progreso durante el entrenamiento.


In [4]:
history_mse = model_mse.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=32,
    verbose=1
)



Epoch 1/30
2211/2211 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - loss: 6745864704.0000 - mae: 31081.7363 - val_loss: 4124099584.0000 - val_mae: 18953.3008
Epoch 2/30
2211/2211 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 3424684032.0000 - mae: 18916.3613 - val_loss: 3226230528.0000 - val_mae: 19320.9668
Epoch 3/30
2211/2211 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - loss: 3041987328.0000 - mae: 19306.5996 - val_loss: 2997132544.0000 - val_mae: 18198.3027
Epoch 4/30
2211/2211 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - loss: 3811432192.0000 - mae: 18718.4512 - val_loss: 2820739840.0000 - val_mae: 16457.1445
Epoch 5/30
2211/2211 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 2548855040.0000 - mae: 16248.0801 - val_loss: 2658615552.0000 - val_mae: 15072.8945
Epoch 6/30
2211/2211 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - loss: 2330080000.0000 - mae: 14931.2861 - val_loss: 2512652032.0000 - val_mae: 13684.7275
Epoch 7/30
2211/2211 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - loss: 1873929088.0000 - mae: 13285.6709 - val_loss: 2379894784.000

* Creamos un grafico para mostrar como evoluciona la perdida del modelo a lo largo de las epocas.
* Añadimos dos lineas:
  - La perdida en los datos de entrenamiento ("loss").
  - La perdida en los datos de validación ("val_loss").
* Esto nos ayuda a ver si el modelo esta aprendiendo bien o si esta sobreajustando.
* El eje X muestra las epocas y el eje Y la magnitud de la perdida (MSE).


In [5]:
fig_mse = go.Figure()
fig_mse.add_trace(go.Scatter(y=history_mse.history['loss'], name='Entrenamiento (MSE)'))
fig_mse.add_trace(go.Scatter(y=history_mse.history['val_loss'], name='Validación (MSE)'))
fig_mse.update_layout(title='Pérdida con función MSE', xaxis_title='Épocas', yaxis_title='MSE')
fig_mse.show()

* En el gráfico, tanto la perdida en entrenamiento como en validacion bajan rapidamente y se acercan bastante desde las primeras epocas. Esto indica que el modelo esta aprendiendo de manera eficiente y generaliza bien, es decir, no solo memoriza los datos de entrenamiento, sino que tambien funciona bien con datos nuevos. La convergencia rapida y cercana de ambas curvas sugiere que el modelo esta bien ajustado, sin problemas evidentes de sobreajuste o subajuste.

Repetimos el mismo proceso de creacion, entrenamiento y evaluacion del modelo, pero esta vez usando la funcion de perdida MAE para observar como se comporta la curva de error absoluto medio durante el entrenamiento y la validacion

In [6]:
model_mae = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)
])

model_mae.compile(optimizer=Adam(), loss='mae', metrics=['mse'])


In [7]:
history_mae = model_mae.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=32,
    verbose=1
)


Epoch 1/30
2211/2211 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 28347.7559 - mse: 5630346240.0000 - val_loss: 11506.3691 - val_mse: 3653378560.0000
Epoch 2/30
2211/2211 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 10492.6973 - mse: 3042323968.0000 - val_loss: 8004.3271 - val_mse: 2485285120.0000
Epoch 3/30
2211/2211 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - loss: 7883.2100 - mse: 1922841472.0000 - val_loss: 6517.7573 - val_mse: 1884075520.0000
Epoch 4/30
2211/2211 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 6249.8945 - mse: 955870464.0000 - val_loss: 5022.3984 - val_mse: 1386573696.0000
Epoch 5/30
2211/2211 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 4602.1948 - mse: 508127360.0000 - val_loss: 3972.9448 - val_mse: 1062010432.0000
Epoch 6/30
2211/2211 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - loss: 3695.0457 - mse: 257411728.0000 - val_loss: 3582.4446 - val_mse: 934904064.0000
Epoch 7/30
2211/2211 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - loss: 3450.9590 - mse: 383357632.0000 - val_loss: 3465.3252 - val_mse: 904706304.

In [8]:
os.makedirs('metricas', exist_ok=True)

with open('metricas/metricas_modelo_dl.pkl', 'wb') as f:
    pickle.dump(history_mae.history, f)


In [9]:
fig_mae = go.Figure()
fig_mae.add_trace(go.Scatter(y=history_mae.history['loss'], name='Entrenamiento (MAE)'))
fig_mae.add_trace(go.Scatter(y=history_mae.history['val_loss'], name='Validación (MAE)'))
fig_mae.update_layout(title='Pérdida con función MAE', xaxis_title='Épocas', yaxis_title='MAE')
fig_mae.show()


* En este grafico, ambas lineas —entrenamiento y validacion— se acercan y disminuyen juntas de manera constante, lo que indica que el modelo esta aprendiendo correctamente y no presenta sobreajuste (overfitting). Esto significa que el modelo generaliza bien y su rendimiento es estable tanto con los datos de entrenamiento como con datos nuevos.


* Finalmente guardamos el modelo 

In [10]:
model_mae.save('modelos/modelo_red_neuronal_mae.keras')
